
Autograd: tự động lấy đạo hàm
===================================

Tensor
--------

Nếu ta chọn ``.requires_grad`` là ``True`` trong ``torch.Tensor``thì torch tensor sẽ ghi lại toàn bộ các phép tính đã được thực hiện trên đó. Sau khi tính toán, ta có thể dùng ``.backward()`` để các gradient được tính tự động. Gradient cho tensor này sẽ được tích vào ``.grad``.

để tensor không lưu lại lịch sữ, ta dùng ``.detach()`` để tách nó khỏi lịch sử tính và không lưu các tính toán trong tương lai.

Nếu muốn tính đạo hàm, ta có thể gọi ``.backward()`` trên một ``Tensor``.


In [1]:
import torch

tao tensor, cho ``requires_grad=True`` để track các phép tính: 

In [2]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [3]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward>)


- ``y`` được tạo bởi một phép tính (tức là nó là kết quả của một hàm) nên nó có thuộc tính ``grad_fn``. (``fn`` là viết tắt cho ``function``). 

- Chỉ khi tensor được tạo b3 người dùng thì nó có ``grad_fn is None``.

In [4]:
print(y.grad_fn)

In [5]:
z = y * y * 3
z

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward>)

In [7]:
out = z.mean()
print(z, out)

(tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward>), tensor(27., grad_fn=<MeanBackward1>))


- ``.requires_grad_( ... )`` changes an existing Tensor's ``requires_grad``
flag in-place. The input flag defaults to ``False`` if not given.



In [9]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
a

tensor([[-3.5967,  6.2126],
        [ 5.1532,  1.3799]])

In [10]:
print(a.requires_grad)

False


In [11]:
a.requires_grad_(True)


tensor([[-3.5967,  6.2126],
        [ 5.1532,  1.3799]], requires_grad=True)

In [12]:
print(a.requires_grad)


True


In [13]:
b = (a * a).sum()
print(b.grad_fn)

Gradients
---------
bây giờ chúng ta backprop vì ``out`` chỉ chứa một hằng số, ``out.backward()`` tương đương với ``out.backward(torch.tensor(1))``.



In [14]:
out.backward()

print gradients d(out)/dx




In [15]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


Ta nhan65 được một ma trận toàn ``4.5``. Gọi tensor đầu ra là “$o$”.
ta có $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ và $z_i\bigr\rvert_{x_i=1} = 27$.
 Do đó,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



In [16]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([-446.2421, 1133.3678, 1070.4928], grad_fn=<MulBackward>)


In [17]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([ 102.4000, 1024.0000,    0.1024])


Để autograd ngừng lưu lịch sử trên tensor, ta dùng 
``with torch.no_grad():``



In [18]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

True
True
False


- Tài liệu về ``autograd`` và ``Function``:
http://pytorch.org/docs/autograd

